# 📰 Phase 1 — Financial News Scraper & Preprocessing Pipeline
### Merged Notebook: Yahoo Finance (RSS + HTML) + Multi-Source RSS

**แหล่งข้อมูล:**
- **Yahoo Finance** — RSS Feed (19 feeds) + HTML Scraping (8 หน้า)
- **Multi-Source RSS** — Investing.com, CNBC, Reuters, AP, Forbes, MarketWatch, Seeking Alpha, The Economist, Nasdaq

**Pipeline:**
1. **1.1** Data Collection → รวม raw articles ≥ 500 รายการ (ปี 2025–2026)
2. **1.2** Cleaning → ลบ HTML / URL / Emoji / อักขระพิเศษ → บันทึก `clean_text_(Web_Scraping).csv`
3. **1.2.5** Merge Kaggle Dataset → รวมกับ dataset จาก Kaggle → บันทึก `complete_dataset.csv`
4. **1.3** Tokenization Comparison → NLTK vs spaCy (เปรียบเทียบบน `complete_dataset.csv`)

In [1]:
# ══════════════════════════════════════════════════════════
#  INSTALL DEPENDENCIES
# ══════════════════════════════════════════════════════════
!pip install requests beautifulsoup4 pandas lxml feedparser nltk spacy -q
!python -m spacy download en_core_web_sm -q
print('✅ ติดตั้ง dependencies สำเร็จ')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ ติดตั้ง dependencies สำเร็จ


In [2]:
# ══════════════════════════════════════════════════════════
#  IMPORTS & GLOBAL CONFIG
# ══════════════════════════════════════════════════════════
import requests
from bs4 import BeautifulSoup
import feedparser
import pandas as pd
import time
import re
import random
import json
import os
import logging
from datetime import datetime

import nltk
import spacy

for pkg in ['punkt', 'punkt_tab', 'stopwords', 'wordnet']:
    nltk.download(pkg, quiet=True)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    os.system('python -m spacy download en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

TODAY   = datetime.now().strftime('%Y-%m-%d')
OUT_DIR = 'output_data'
os.makedirs(OUT_DIR, exist_ok=True)

# ชื่อไฟล์ output (เปลี่ยนที่นี่ที่เดียว)
WEB_CSV      = f'{OUT_DIR}/clean_text_(Web_Scraping).csv'
COMPLETE_CSV = f'{OUT_DIR}/complete_dataset.csv'

# Browser-like headers (ป้องกัน 403)
HEADERS = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/120.0.0.0 Safari/537.36'
    ),
    'Accept':          'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer':         'https://finance.yahoo.com/',
}

print(f'✅ Import สำเร็จ | วันที่รันโค้ด: {TODAY}')
print(f'📁 Output dir : {os.path.abspath(OUT_DIR)}')

✅ Import สำเร็จ | วันที่รันโค้ด: 2026-02-19
📁 Output dir : /content/output_data


## Section 1.1 — Data Collection

In [3]:
# ══════════════════════════════════════════════════════════
#  1.1.1  YAHOO FINANCE — RSS FEED SCRAPING
#  ข้อดี: เสถียร 100%, ได้วันที่จริง, ได้ description
# ══════════════════════════════════════════════════════════

YAHOO_RSS_FEEDS = {
    # ข่าวทั่วไป
    'top_stories':  'https://finance.yahoo.com/news/rssindex',
    'latest_news':  'https://finance.yahoo.com/rss/topstories',
    # ดัชนีตลาด
    'sp500':        'https://finance.yahoo.com/rss/headline?s=^GSPC',
    'nasdaq':       'https://finance.yahoo.com/rss/headline?s=^IXIC',
    'dowjones':     'https://finance.yahoo.com/rss/headline?s=^DJI',
    # หุ้น Tech
    'apple':        'https://finance.yahoo.com/rss/headline?s=AAPL',
    'microsoft':    'https://finance.yahoo.com/rss/headline?s=MSFT',
    'nvidia':       'https://finance.yahoo.com/rss/headline?s=NVDA',
    'google':       'https://finance.yahoo.com/rss/headline?s=GOOGL',
    'meta':         'https://finance.yahoo.com/rss/headline?s=META',
    'tesla':        'https://finance.yahoo.com/rss/headline?s=TSLA',
    'amazon':       'https://finance.yahoo.com/rss/headline?s=AMZN',
    # Banking
    'jpmorgan':     'https://finance.yahoo.com/rss/headline?s=JPM',
    'bank_america': 'https://finance.yahoo.com/rss/headline?s=BAC',
    'goldman':      'https://finance.yahoo.com/rss/headline?s=GS',
    # สินทรัพย์อื่น
    'bitcoin':      'https://finance.yahoo.com/rss/headline?s=BTC-USD',
    'ethereum':     'https://finance.yahoo.com/rss/headline?s=ETH-USD',
    'gold':         'https://finance.yahoo.com/rss/headline?s=GC=F',
    'oil':          'https://finance.yahoo.com/rss/headline?s=CL=F',
}


def scrape_yahoo_rss(feed_url: str, category: str) -> list:
    """ดึงข่าวจาก Yahoo Finance RSS — คืน list of dict"""
    try:
        time.sleep(random.uniform(1.0, 2.0))
        resp = requests.get(feed_url, headers=HEADERS, timeout=10)
        if resp.status_code != 200:
            return []
        soup    = BeautifulSoup(resp.text, 'xml')
        results = []
        for item in soup.find_all('item'):
            title    = item.find('title')
            pub_date = item.find('pubDate')
            desc     = item.find('description')
            if not title:
                continue
            headline = title.get_text(strip=True)
            if len(headline) < 20:
                continue
            raw_date = pub_date.get_text(strip=True) if pub_date else TODAY
            try:
                dt         = datetime.strptime(raw_date, '%a, %d %b %Y %H:%M:%S %z')
                clean_date = dt.strftime('%Y-%m-%d')
                year       = dt.year
            except:
                clean_date, year = TODAY, 2025
            content = ''
            if desc:
                content = BeautifulSoup(desc.get_text(), 'html.parser').get_text(strip=True)[:800]
            results.append({
                'title':    headline,
                'content':  content,
                'date':     clean_date,
                'year':     year,
                'source':   'Yahoo Finance',
                'category': category,
            })
        return results
    except:
        return []


print('📡 Yahoo Finance RSS Scraping...')
print('=' * 55)
yahoo_rss_rows = []
for cat, url in YAHOO_RSS_FEEDS.items():
    rows   = scrape_yahoo_rss(url, cat)
    yahoo_rss_rows.extend(rows)
    status = f'✅ {len(rows)} รายการ' if rows else '⚠️  0 รายการ'
    print(f'  {cat:<16} {status}')

df_yahoo_rss = pd.DataFrame(yahoo_rss_rows) if yahoo_rss_rows else pd.DataFrame()
print(f'\n📊 Yahoo RSS รวม: {len(df_yahoo_rss)} รายการ')

📡 Yahoo Finance RSS Scraping...
  top_stories      ✅ 46 รายการ
  latest_news      ✅ 46 รายการ
  sp500            ✅ 20 รายการ
  nasdaq           ✅ 20 รายการ
  dowjones         ✅ 20 รายการ
  apple            ✅ 20 รายการ
  microsoft        ✅ 20 รายการ
  nvidia           ✅ 20 รายการ
  google           ✅ 20 รายการ
  meta             ✅ 19 รายการ
  tesla            ✅ 20 รายการ
  amazon           ✅ 20 รายการ
  jpmorgan         ✅ 20 รายการ
  bank_america     ✅ 20 รายการ
  goldman          ✅ 20 รายการ
  bitcoin          ✅ 20 รายการ
  ethereum         ✅ 20 รายการ
  gold             ✅ 20 รายการ
  oil              ✅ 20 รายการ

📊 Yahoo RSS รวม: 431 รายการ


In [4]:
# ══════════════════════════════════════════════════════════
#  1.1.2  YAHOO FINANCE — HTML SCRAPING (เสริมปริมาณ)
# ══════════════════════════════════════════════════════════

YAHOO_HTML_URLS = {
    'news_main':   'https://finance.yahoo.com/news/',
    'markets':     'https://finance.yahoo.com/markets/',
    'stocks':      'https://finance.yahoo.com/topic/stock-market-news/',
    'crypto':      'https://finance.yahoo.com/crypto/',
    'economy':     'https://finance.yahoo.com/topic/economic-news/',
    'earnings':    'https://finance.yahoo.com/topic/earnings/',
    'tech':        'https://finance.yahoo.com/topic/tech/',
    'real_estate': 'https://finance.yahoo.com/topic/real-estate/',
}


def scrape_yahoo_html(url: str, category: str, seen: set) -> list:
    """Scrape headline จากหน้า HTML Yahoo Finance"""
    try:
        time.sleep(random.uniform(2.0, 3.5))
        resp = requests.get(url, headers=HEADERS, timeout=15)
        if resp.status_code != 200:
            return []
        soup    = BeautifulSoup(resp.text, 'html.parser')
        results = []
        for tag in soup.find_all(['h1', 'h2', 'h3']):
            text = tag.get_text(strip=True)
            if len(text) <= 25 or text in seen:
                continue
            seen.add(text)
            parent  = tag.parent
            p_tag   = parent.find('p') if parent else None
            content = p_tag.get_text(strip=True)[:800] if p_tag else text
            results.append({
                'title':    text,
                'content':  content,
                'date':     TODAY,
                'year':     2025,
                'source':   'Yahoo Finance',
                'category': category,
            })
        return results
    except:
        return []


print('🌐 Yahoo Finance HTML Scraping...')
print('=' * 55)
seen_yahoo      = set(df_yahoo_rss['title'].tolist() if not df_yahoo_rss.empty else [])
yahoo_html_rows = []
for cat, url in YAHOO_HTML_URLS.items():
    rows   = scrape_yahoo_html(url, cat, seen_yahoo)
    yahoo_html_rows.extend(rows)
    status = f'✅ {len(rows)} รายการ' if rows else '⚠️  0 รายการ'
    print(f'  {cat:<15} {status}')

df_yahoo_html = pd.DataFrame(yahoo_html_rows) if yahoo_html_rows else pd.DataFrame()
print(f'\n📊 Yahoo HTML รวม: {len(df_yahoo_html)} รายการ')

🌐 Yahoo Finance HTML Scraping...
  news_main       ✅ 1 รายการ
  markets         ✅ 1 รายการ
  stocks          ✅ 44 รายการ
  crypto          ⚠️  0 รายการ
  economy         ✅ 49 รายการ
  earnings        ✅ 52 รายการ
  tech            ✅ 53 รายการ
  real_estate     ⚠️  0 รายการ

📊 Yahoo HTML รวม: 200 รายการ


In [5]:
# ══════════════════════════════════════════════════════════
#  1.1.3  MULTI-SOURCE RSS SCRAPING
#  แหล่ง: Investing.com, CNBC, Reuters, AP, Forbes,
#          MarketWatch, Seeking Alpha, The Economist, Nasdaq
# ══════════════════════════════════════════════════════════

MULTI_RSS_FEEDS = [
    # Investing.com
    {'source': 'Investing.com - Stock Market', 'url': 'https://www.investing.com/rss/news_285.rss'},
    {'source': 'Investing.com - Economy',      'url': 'https://www.investing.com/rss/news_1.rss'},
    {'source': 'Investing.com - Forex',        'url': 'https://www.investing.com/rss/news_25.rss'},
    {'source': 'Investing.com - Commodities',  'url': 'https://www.investing.com/rss/news_8.rss'},
    {'source': 'Investing.com - Crypto',       'url': 'https://www.investing.com/rss/news_301.rss'},
    # CNBC
    {'source': 'CNBC - Top News',      'url': 'https://www.cnbc.com/id/100003114/device/rss/rss.html'},
    {'source': 'CNBC - Finance',       'url': 'https://www.cnbc.com/id/10000664/device/rss/rss.html'},
    {'source': 'CNBC - Earnings',      'url': 'https://www.cnbc.com/id/15839135/device/rss/rss.html'},
    {'source': 'CNBC - Markets',       'url': 'https://www.cnbc.com/id/20910258/device/rss/rss.html'},
    {'source': 'CNBC - Economy',       'url': 'https://www.cnbc.com/id/20910274/device/rss/rss.html'},
    {'source': 'CNBC - World Markets', 'url': 'https://www.cnbc.com/id/15839069/device/rss/rss.html'},
    # Reuters
    {'source': 'Reuters - Business', 'url': 'https://feeds.reuters.com/reuters/businessNews'},
    {'source': 'Reuters - Markets',  'url': 'https://feeds.reuters.com/reuters/companyNews'},
    {'source': 'Reuters - US',       'url': 'https://feeds.reuters.com/Reuters/domesticNews'},
    # AP News
    {'source': 'AP - Business', 'url': 'https://rsshub.app/apnews/topics/business'},
    {'source': 'AP - Economy',  'url': 'https://rsshub.app/apnews/topics/economy'},
    {'source': 'AP - Finance',  'url': 'https://rsshub.app/apnews/topics/financial-markets'},
    # Forbes
    {'source': 'Forbes - Business',  'url': 'https://www.forbes.com/business/feed/'},
    {'source': 'Forbes - Markets',   'url': 'https://www.forbes.com/money/feed/'},
    {'source': 'Forbes - Investing', 'url': 'https://www.forbes.com/investing/feed/'},
    # MarketWatch
    {'source': 'MarketWatch - Top Stories',  'url': 'https://feeds.content.dowjones.io/public/rss/mw_topstories'},
    {'source': 'MarketWatch - Market Pulse', 'url': 'https://feeds.content.dowjones.io/public/rss/mw_marketpulse'},
    {'source': 'MarketWatch - Real Estate',  'url': 'https://feeds.content.dowjones.io/public/rss/mw_realestate'},
    # Seeking Alpha
    {'source': 'Seeking Alpha - Market News', 'url': 'https://seekingalpha.com/market_currents.xml'},
    {'source': 'Seeking Alpha - Wall Street', 'url': 'https://seekingalpha.com/feed/wall-street-breakfast'},
    # Barron's
    {'source': "Barron's", 'url': 'https://www.barrons.com/xml/rss/3_7510.xml'},
    # The Economist
    {'source': 'The Economist - Finance', 'url': 'https://www.economist.com/finance-and-economics/rss.xml'},
    {'source': 'The Economist - Business','url': 'https://www.economist.com/business/rss.xml'},
    # Nasdaq
    {'source': 'Nasdaq - Markets',  'url': 'https://www.nasdaq.com/feed/rssoutbound?category=Markets'},
    {'source': 'Nasdaq - Original', 'url': 'https://www.nasdaq.com/feed/rssoutbound?category=Original+Articles'},
]


class RSSFinancialScraper:
    """ดึงข่าวจาก Multi-Source RSS ด้วย feedparser"""

    def __init__(self, target: int = 700, fetch_full_article: bool = True,
                 delay: tuple = (0.5, 1.5)):
        self.target      = target
        self.fetch_full  = fetch_full_article
        self.delay       = delay
        self.articles    = []
        self.seen_titles = set()

    def _sleep(self):
        time.sleep(random.uniform(*self.delay))

    def _extract_full_content(self, url: str) -> str:
        """ดึงเนื้อหาเต็มจาก article URL"""
        try:
            self._sleep()
            r    = requests.get(url, headers=HEADERS, timeout=12)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, 'html.parser')
            for sel in [
                {'tag': 'article'},
                {'tag': 'div', 'class_': re.compile(r'article.body|story.body|content.body|post.body', re.I)},
                {'tag': 'div', 'id':     re.compile(r'article|content|main', re.I)},
            ]:
                tag_name = sel.pop('tag')
                el       = soup.find(tag_name, **sel)
                if el:
                    text = ' '.join(p.get_text(' ', strip=True) for p in el.find_all('p'))
                    if len(text.split()) > 30:
                        return text
            return ' '.join(p.get_text(' ', strip=True) for p in soup.find_all('p'))
        except Exception as e:
            logger.debug(f'[HTML ERROR] {url[:60]} → {e}')
            return ''

    def _parse_rss(self, feed_url: str, source: str) -> int:
        """Parse RSS feed ด้วย feedparser"""
        added = 0
        try:
            feed = feedparser.parse(feed_url, agent=HEADERS['User-Agent'])
            if feed.bozo and not feed.entries:
                return 0
            for entry in feed.entries:
                if len(self.articles) >= self.target:
                    break
                title = getattr(entry, 'title', '').strip()
                if not title or title in self.seen_titles:
                    continue
                self.seen_titles.add(title)
                url = getattr(entry, 'link', '')
                # Content: content > summary > description
                content = ''
                if hasattr(entry, 'content') and entry.content:
                    content = entry.content[0].get('value', '')
                if not content and hasattr(entry, 'summary'):
                    content = entry.summary
                if not content and hasattr(entry, 'description'):
                    content = entry.description
                content = BeautifulSoup(content, 'html.parser').get_text(' ', strip=True)
                # fetch full article ถ้า content สั้น
                if self.fetch_full and url and len(content.split()) < 80:
                    full = self._extract_full_content(url)
                    if full:
                        content = full
                if not content or len(content.split()) < 10:
                    continue
                pub = getattr(entry, 'published', getattr(entry, 'updated', ''))
                try:
                    from email.utils import parsedate_to_datetime
                    dt         = parsedate_to_datetime(pub)
                    clean_date = dt.strftime('%Y-%m-%d')
                    year       = dt.year
                except:
                    clean_date, year = TODAY, 2025
                self.articles.append({
                    'title':    title,
                    'content':  content[:1000],
                    'date':     clean_date,
                    'year':     year,
                    'source':   source,
                    'category': source.split(' - ')[-1].lower().replace(' ', '_'),
                })
                added += 1
        except Exception as e:
            logger.warning(f'[RSS ERROR] {source}: {e}')
        return added

    def collect(self) -> pd.DataFrame:
        logger.info(f'🚀 Multi-source RSS เป้าหมาย {self.target} รายการ จาก {len(MULTI_RSS_FEEDS)} feeds')
        for i, feed in enumerate(MULTI_RSS_FEEDS):
            if len(self.articles) >= self.target:
                break
            n = self._parse_rss(feed['url'], feed['source'])
            logger.info(f'  [{i+1:>2}/{len(MULTI_RSS_FEEDS)}] {feed["source"]:<35} +{n} | รวม {len(self.articles)}/{self.target}')
        logger.info(f'✅ รวบรวมได้ {len(self.articles)} รายการ')
        return pd.DataFrame(self.articles)


scraper  = RSSFinancialScraper(target=700, fetch_full_article=True, delay=(0.5, 1.5))
df_multi = scraper.collect()
print(f'\n📊 Multi-Source RSS รวม: {len(df_multi)} รายการ')


📊 Multi-Source RSS รวม: 489 รายการ


In [6]:
# ══════════════════════════════════════════════════════════
#  1.1.4  รวม Dataset ทั้งหมด + กรอง 2025–2026
# ══════════════════════════════════════════════════════════

all_dfs = [df for df in [df_yahoo_rss, df_yahoo_html, df_multi] if not df.empty]

if all_dfs:
    df_raw = pd.concat(all_dfs, ignore_index=True)
    df_raw = df_raw.drop_duplicates(subset=['title']).reset_index(drop=True)
    df_raw = df_raw[df_raw['year'].isin([2025, 2026])].reset_index(drop=True)
else:
    # ─── Fallback Demo Dataset ─────────────────────────────
    print('⚠️  Scraping ไม่ได้ผล — ใช้ Demo Dataset แทน')
    df_raw = pd.DataFrame([
        {'title': 'Apple reports record Q1 2025 revenue beating all forecasts',
         'content': 'Apple Inc. surpassed Wall Street expectations with record-breaking Q1 2025 revenue driven by strong iPhone and services growth.',
         'date': '2025-02-10', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'apple'},
        {'title': 'Federal Reserve keeps interest rates unchanged in February 2025',
         'content': 'The Federal Reserve held its benchmark rate steady, signaling caution amid mixed economic data and persistent inflation concerns.',
         'date': '2025-02-01', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'top_stories'},
        {'title': 'Nvidia stock surges 8 percent after strong earnings guidance',
         'content': 'Nvidia Corporation shares jumped sharply following better-than-expected earnings and bullish guidance tied to AI chip demand.',
         'date': '2025-02-12', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'nvidia'},
        {'title': 'Bitcoin falls below 90000 as crypto market sees correction',
         'content': 'Cryptocurrency markets experienced a sharp decline with Bitcoin dropping below the 90,000 dollar mark amid profit-taking.',
         'date': '2025-02-05', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'bitcoin'},
        {'title': 'Tesla shares drop 12 percent on weak delivery numbers',
         'content': 'Tesla Inc. reported lower-than-expected deliveries for Q1 2025, disappointing investors and triggering a sharp stock selloff.',
         'date': '2025-02-03', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'tesla'},
        {'title': 'S&P 500 hits new record as tech stocks lead market rally',
         'content': 'The benchmark S&P 500 index closed at a new record high, powered by broad gains in mega-cap technology stocks.',
         'date': '2025-02-14', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'sp500'},
        {'title': 'JPMorgan raises dividend after strong fourth quarter earnings',
         'content': 'JPMorgan Chase announced an increase to its quarterly dividend following a robust Q4 2024 earnings report that beat analyst estimates.',
         'date': '2025-01-20', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'jpmorgan'},
        {'title': 'Oil prices rise on OPEC production cut agreement for 2025',
         'content': 'Crude oil futures climbed higher after OPEC+ members agreed to extend production cuts through mid-2025 to support prices.',
         'date': '2025-01-15', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'oil'},
        {'title': 'Gold hits all-time high as investors seek safe haven assets',
         'content': 'Gold futures surged to a record high amid heightened geopolitical tensions and growing demand for safe-haven assets globally.',
         'date': '2025-02-08', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'gold'},
        {'title': 'US inflation drops to 2.5 percent lowest level since 2021',
         'content': 'Consumer prices rose at a slower pace in January 2025, bringing inflation to its lowest point since 2021 and bolstering rate-cut hopes.',
         'date': '2025-02-13', 'year': 2025, 'source': 'Yahoo Finance', 'category': 'economy'},
    ])

print(f'✅ รวม Dataset สำเร็จ')
print(f'   จำนวนทั้งหมด : {len(df_raw):,} รายการ')
print(f'   แหล่งข้อมูล  : {df_raw["source"].nunique()} แหล่ง')
print(f'   ปีที่มีข้อมูล: {sorted(df_raw["year"].unique().tolist())}')
df_raw.head(3)

✅ รวม Dataset สำเร็จ
   จำนวนทั้งหมด : 956 รายการ
   แหล่งข้อมูล  : 14 แหล่ง
   ปีที่มีข้อมูล: [2025, 2026]


,title,content,date,year,source,category
0,Hackett Group HCKT Earnings Call Transcript,,2026-02-19,2025,Yahoo Finance,top_stories
1,Labcorp Holdings Inc. Q4 2025 Earnings Call Su...,,2026-02-19,2025,Yahoo Finance,top_stories
2,ReNew Energy Global Plc Q3 2026 Earnings Call ...,,2026-02-19,2025,Yahoo Finance,top_stories


## Section 1.2 — Cleaning Pipeline → บันทึก `clean_text_(Web_Scraping).csv`

In [7]:
# ══════════════════════════════════════════════════════════
#  1.2  CLEANING PIPELINE
#  HTML tags → URLs → Emojis → Special chars → Whitespace
#  จากนั้น build Sentence (title ขึ้นต้น + content)
#  และบันทึก clean_text_(Web_Scraping).csv
# ══════════════════════════════════════════════════════════

EMOJI_RE = re.compile(
    '['
    u'\U0001F600-\U0001F64F'
    u'\U0001F300-\U0001F5FF'
    u'\U0001F680-\U0001F6FF'
    u'\U0001F1E0-\U0001F1FF'
    u'\U00002702-\U000027B0'
    u'\U000024C2-\U0001F251'
    ']+', flags=re.UNICODE
)


def clean_field(text: str) -> str:
    """
    ทำความสะอาด 1 field:
    1. ลบ HTML tags   (BeautifulSoup)
    2. ลบ URLs        (http/https/www)
    3. ลบ Emojis      (Unicode ranges)
    4. ลบ special chars (เก็บ a-z A-Z 0-9 .,!?;:'\"%-+&)
    5. Normalize whitespace
    """
    if not isinstance(text, str) or not text.strip():
        return ''
    text = BeautifulSoup(text, 'html.parser').get_text(separator=' ')   # 1
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)               # 2
    text = EMOJI_RE.sub(' ', text)                                       # 3
    text = re.sub(r"[^a-zA-Z0-9\s.,!?;:'\"&%+-]", ' ', text)         # 4
    text = re.sub(r'\s+', ' ', text).strip()                            # 5
    return text


def build_sentence(title: str, content: str) -> str:
    """
    รวม title + content เป็น Sentence เดียว
    - title ต้องขึ้นต้นเสมอ  (ห้ามแทรกตำแหน่งอื่น)
    - ถ้า content ไม่ว่าง: '<title>. <content>'
    - ถ้า content ซ้ำกับ title (บางแหล่ง copy title เป็น content): ใช้แค่ title
    - normalize whitespace รอบสุดท้าย
    """
    title   = str(title).strip()
    content = str(content).strip()

    if title and content:
        if content.lower().startswith(title.lower()):
            sentence = content
        else:
            sep      = '' if title.endswith('.') else '. '
            sentence = title + sep + content
    elif title:
        sentence = title
    else:
        return ''

    # Final normalize
    sentence = re.sub(r'[ \t]+', ' ', sentence)
    sentence = re.sub(r'\n+',    ' ', sentence)
    return sentence.strip()


# ─── Apply cleaning ───────────────────────────────────────
df_clean = df_raw.copy()
df_clean['title_clean']   = df_clean['title'].fillna('').apply(clean_field)
df_clean['content_clean'] = df_clean['content'].fillna('').apply(clean_field)
df_clean['word_count']    = df_clean['content_clean'].str.split().str.len()

# ลบแถวที่ title ว่าง หรือ content < 10 คำ
before   = len(df_clean)
df_clean = df_clean[
    (df_clean['title_clean'].str.len() > 0) &
    (df_clean['content_clean'].str.len() > 0) &
    (df_clean['word_count'] >= 10)
].reset_index(drop=True)

print('✅ Cleaning Pipeline เสร็จสิ้น')
print(f'   ก่อน clean : {before:,} รายการ')
print(f'   หลัง clean : {len(df_clean):,} รายการ')
print(f'   ตัดออก     : {before - len(df_clean):,} รายการ (ว่าง / สั้นเกิน)')

# ─── Build Sentence ───────────────────────────────────────
df_clean['Sentence'] = df_clean.apply(
    lambda r: build_sentence(r['title_clean'], r['content_clean']), axis=1
)

# ─── สร้าง Web Scraping CSV (Sentence column เดียว) ──────
df_web = (
    df_clean[['Sentence']]
    .copy()
    .assign(Sentence=lambda d: (
        d['Sentence']
        .str.replace(r'[ \t]+', ' ', regex=True)
        .str.replace(r'\n+',    ' ', regex=True)
        .str.strip()
    ))
)
# ลบแถวที่ Sentence ว่างหรือสั้นเกิน
df_web = df_web[df_web['Sentence'].str.split().str.len() >= 5].reset_index(drop=True)

# บันทึก
df_web.to_csv(WEB_CSV, index=False, encoding='utf-8-sig')

print(f'\n📊 Web Scraping Dataset')
print(f'   จำนวน Sentence : {len(df_web):,} รายการ')
print(f'💾 บันทึก → {WEB_CSV}')
print()
print('--- ตัวอย่าง 3 Sentence แรก ---')
for i, row in df_web.head(3).iterrows():
    print(f'[{i+1}] {row["Sentence"][:180]}')
    print()

✅ Cleaning Pipeline เสร็จสิ้น
   ก่อน clean : 956 รายการ
   หลัง clean : 839 รายการ
   ตัดออก     : 117 รายการ (ว่าง / สั้นเกิน)

📊 Web Scraping Dataset
   จำนวน Sentence : 839 รายการ
💾 บันทึก → output_data/clean_text_(Web_Scraping).csv

--- ตัวอย่าง 3 Sentence แรก ---
[1] Three Top Dividend Stocks To Consider. As February begins, U.S. stock indexes have shown strength, with the Dow Jones Industrial Average adding 515 points and the S&P 500 nearing a

[2] Dow Jones Futures Fall After S&P 500 Hits Resistance; Carvana Dives, Walmart Due. Futures fall after the stock market extended a bounce Wednesday, but the S&P 500 hit resistance. C

[3] Here's Why I Wouldn't Touch Medical Properties Trust With a 10 Foot Pole. Medical Properties Trust has a huge 6.6% yield, but the dividend history is a big problem for me.



## Section 1.2.5 — Merge Kaggle Dataset → `complete_dataset.csv`

> **วิธีใช้:** รันเซลล์ถัดไป แล้วกดปุ่ม **"📂 Browse Kaggle CSV"** เพื่อเลือกไฟล์ dataset จาก Kaggle ในเครื่องของคุณ  
> โค้ดจะตัด column ที่ไม่จำเป็นออกอัตโนมัติ เหลือเฉพาะ `Sentence` แล้ว merge รวมกับ Web Scraping dataset

In [15]:
# ══════════════════════════════════════════════════════════
#  1.2.5  MERGE KAGGLE DATASET (Google Colab version)
#  Upload file → ตัด column เหลือแค่ Sentence → merge
# ══════════════════════════════════════════════════════════

from google.colab import files
from IPython.display import display
import io

# ─── Helper: หา column ที่น่าจะเป็น Sentence ─────────────
def _find_sentence_col(df):
    """
    ลำดับความสำคัญในการหา column ที่เป็นเนื้อข่าว:
    1. ชื่อ 'Sentence' ตรงๆ (case-insensitive)
    2. ชื่อ 'text', 'content', 'article', 'body', 'description', 'news'
    3. column string ที่มีความยาวเฉลี่ยมากที่สุด (fallback)
    """
    col_lower = {c.lower(): c for c in df.columns}
    for keyword in ['sentence', 'text', 'content', 'article', 'body', 'description', 'news']:
        if keyword in col_lower:
            return col_lower[keyword]
    # fallback: column string ที่ยาวที่สุดโดยเฉลี่ย
    str_cols = df.select_dtypes(include='object').columns.tolist()
    if str_cols:
        avg_len = {c: df[c].astype(str).str.len().mean() for c in str_cols}
        return max(avg_len, key=avg_len.get)
    return None


# ─── Step 1: เลือกไฟล์ CSV จากเครื่อง ───────────────────
print('📂 กรุณาเลือกไฟล์ Kaggle dataset (.csv) จากเครื่องของคุณ')
uploaded = files.upload()   # จะเปิด file picker ของ Colab ขึ้นมาทันที

if not uploaded:
    print('❌ ไม่ได้เลือกไฟล์ หยุดการทำงาน')
else:
    # ─── Step 2: อ่านไฟล์ที่อัปโหลด ─────────────────────
    fname    = list(uploaded.keys())[0]
    raw_bytes = uploaded[fname]

    print(f'\n📄 ไฟล์ที่เลือก: {fname}  ({len(raw_bytes)/1024:.1f} KB)')

    try:
        df_kaggle_raw = pd.read_csv(io.BytesIO(raw_bytes), encoding='utf-8-sig')
    except UnicodeDecodeError:
        df_kaggle_raw = pd.read_csv(io.BytesIO(raw_bytes), encoding='latin-1')

    print(f'📋 Columns ทั้งหมด : {list(df_kaggle_raw.columns)}')
    print(f'   จำนวนแถว       : {len(df_kaggle_raw):,}')

    # ─── Step 3: หา column เนื้อข่าวอัตโนมัติ ────────────
    src_col = _find_sentence_col(df_kaggle_raw)
    if src_col is None:
        print('❌ ไม่พบ column ที่เหมาะสม กรุณาตรวจสอบไฟล์')
    else:
        print(f'\n🔍 เลือกใช้ column : "{src_col}" เป็น Sentence')

        # ─── Step 4: ตัดเหลือแค่ Sentence + clean ────────
        df_kaggle = pd.DataFrame({'Sentence': df_kaggle_raw[src_col].astype(str)})
        df_kaggle['Sentence'] = (
            df_kaggle['Sentence']
            .apply(clean_field)                         # clean_field จาก Section 1.2
            .str.replace(r'[ \t]+', ' ', regex=True)
            .str.replace(r'\n+',    ' ', regex=True)
            .str.strip()
        )
        # ลบแถวที่ว่าง หรือ < 5 คำ
        df_kaggle = df_kaggle[
            df_kaggle['Sentence'].str.strip().str.len() > 0
        ].reset_index(drop=True)
        df_kaggle = df_kaggle[
            df_kaggle['Sentence'].str.split().str.len() >= 5
        ].reset_index(drop=True)

        print(f'   หลัง clean     : {len(df_kaggle):,} รายการ')

        # ─── Step 5: Merge รวมกับ Web Scraping ───────────
        df_complete = pd.concat([df_web, df_kaggle], ignore_index=True)
        df_complete = df_complete.drop_duplicates(subset=['Sentence']).reset_index(drop=True)
        df_complete['Sentence'] = (
            df_complete['Sentence']
            .str.replace(r'[ \t]+', ' ', regex=True)
            .str.replace(r'\n+',    ' ', regex=True)
            .str.strip()
        )

        # ─── Step 6: บันทึก complete_dataset.csv ─────────
        df_complete.to_csv(COMPLETE_CSV, index=False, encoding='utf-8-sig')

        print()
        print('=' * 60)
        print('✅ Merge Dataset สำเร็จ!')
        print(f'   Web Scraping : {len(df_web):,} รายการ')
        print(f'   Kaggle       : {len(df_kaggle):,} รายการ')
        print(f'   รวม (dedup) : {len(df_complete):,} รายการ')
        print(f'💾 บันทึก → {COMPLETE_CSV}')
        print()
        print('--- ตัวอย่าง 5 Sentence แรก ---')
        for i, row in df_complete.head(5).iterrows():
            print(f'[{i+1}] {row["Sentence"][:180]}')
            print()

📂 กรุณาเลือกไฟล์ Kaggle dataset (.csv) จากเครื่องของคุณ


Saving Financial Sentiment Analysis 1.csv to Financial Sentiment Analysis 1.csv

📄 ไฟล์ที่เลือก: Financial Sentiment Analysis 1.csv  (727.7 KB)
📋 Columns ทั้งหมด : ['Sentence', 'Sentiment']
   จำนวนแถว       : 5,842

🔍 เลือกใช้ column : "Sentence" เป็น Sentence
   หลัง clean     : 5,781 รายการ

✅ Merge Dataset สำเร็จ!
   Web Scraping : 839 รายการ
   Kaggle       : 5,781 รายการ
   รวม (dedup) : 6,095 รายการ
💾 บันทึก → output_data/complete_dataset.csv

--- ตัวอย่าง 5 Sentence แรก ---
[1] Three Top Dividend Stocks To Consider. As February begins, U.S. stock indexes have shown strength, with the Dow Jones Industrial Average adding 515 points and the S&P 500 nearing a

[2] Dow Jones Futures Fall After S&P 500 Hits Resistance; Carvana Dives, Walmart Due. Futures fall after the stock market extended a bounce Wednesday, but the S&P 500 hit resistance. C

[3] Here's Why I Wouldn't Touch Medical Properties Trust With a 10 Foot Pole. Medical Properties Trust has a huge 6.6% yield, but the dividen

In [16]:
# ══════════════════════════════════════════════════════════
#  โหลด complete_dataset.csv สำหรับส่งต่อ Section 1.3
#  (รันเซลล์นี้หลังจาก Merge เสร็จแล้ว)
# ══════════════════════════════════════════════════════════

if os.path.exists(COMPLETE_CSV):
    df_complete = pd.read_csv(COMPLETE_CSV, encoding='utf-8-sig')
    print(f'✅ โหลด complete_dataset.csv สำเร็จ')
    print(f'   จำนวน Sentence : {len(df_complete):,} รายการ')
    print(f'   Column         : {list(df_complete.columns)}')
else:
    # ยังไม่ได้ merge → ใช้ Web Scraping อย่างเดียวก่อน (fallback)
    print('⚠️  ยังไม่พบ complete_dataset.csv')
    print('    → กรุณารัน Section 1.2.5 (Merge Kaggle) ก่อน')
    print('    → ใช้ Web Scraping dataset แทนชั่วคราว')
    df_complete = df_web.copy()

# ── ลบไฟล์ที่ไม่ต้องการออกจาก output_data ───────────────
_unwanted = ['raw_news.csv', 'processed_news.csv']
for fname in _unwanted:
    fpath = f'{OUT_DIR}/{fname}'
    if os.path.exists(fpath):
        os.remove(fpath)
        print(f'🗑️  ลบ {fname} แล้ว')

# ── แสดงไฟล์ที่มีใน output_data ──────────────────────────
print(f'\n📁 ไฟล์ใน {OUT_DIR}/:')
for f in sorted(os.listdir(OUT_DIR)):
    if f.endswith('.csv') and not f.startswith('_'):
        size = os.path.getsize(f'{OUT_DIR}/{f}') / 1024
        print(f'   ✅ {f}  ({size:.1f} KB)')

df_complete.head(3)

✅ โหลด complete_dataset.csv สำเร็จ
   จำนวน Sentence : 6,095 รายการ
   Column         : ['Sentence']

📁 ไฟล์ใน output_data/:
   ✅ clean_text_(Web_Scraping).csv  (322.1 KB)
   ✅ complete_dataset.csv  (923.7 KB)


,Sentence
0,Three Top Dividend Stocks To Consider. As Febr...
1,Dow Jones Futures Fall After S&P 500 Hits Resi...
2,Here's Why I Wouldn't Touch Medical Properties...


## Section 1.3 — Tokenization Comparison (NLTK vs spaCy)

เปรียบเทียบการตัดคำ 2 วิธี บน `complete_dataset.csv`:
- **NLTK** `word_tokenize` + stopword removal (word-level, regex-based, เร็ว)
- **spaCy** lemmatize + stopword removal (statistical model, ละเอียดกว่า)

In [17]:
# ══════════════════════════════════════════════════════════
#  1.3  TOKENIZATION COMPARISON — NLTK vs spaCy
#  ประมวลผลบน complete_dataset.csv
# ══════════════════════════════════════════════════════════

import time as _time

STOPWORDS_EN = set(stopwords.words('english'))


def tok_nltk(text: str) -> list:
    """Word-level: NLTK word_tokenize + stopword removal"""
    tokens = word_tokenize(str(text).lower())
    return [t for t in tokens if t.isalpha() and t not in STOPWORDS_EN]


def tok_spacy(text: str) -> list:
    """Word-level: spaCy lemmatize + stopword removal"""
    doc = nlp(str(text)[:50_000])
    return [tok.lemma_.lower() for tok in doc
            if tok.is_alpha and not tok.is_stop and len(tok.text) > 1]


# ── เปรียบเทียบบน 5 ข่าวแรก ────────────────────────────
sample_texts = df_complete['Sentence'].head(5).tolist()
print('🔤 Tokenization Comparison: NLTK vs spaCy')
print('=' * 65)

comparison_rows = []
for i, text in enumerate(sample_texts):
    t0      = _time.time()
    n_toks  = tok_nltk(text)
    nltk_ms = (_time.time() - t0) * 1000

    t0      = _time.time()
    s_toks  = tok_spacy(text)
    spacy_ms= (_time.time() - t0) * 1000

    comparison_rows.append({
        'text_preview':  text[:60] + ('...' if len(text) > 60 else ''),
        'nltk_count':    len(n_toks),
        'spacy_count':   len(s_toks),
        'nltk_time_ms':  round(nltk_ms,  2),
        'spacy_time_ms': round(spacy_ms, 2),
        'nltk_sample':   n_toks[:8],
        'spacy_sample':  s_toks[:8],
    })
    print(f'\n[{i+1}] {text[:70]}')
    print(f'    NLTK  ({len(n_toks):>3} tokens, {nltk_ms:5.1f}ms): {n_toks[:8]}')
    print(f'    spaCy ({len(s_toks):>3} tokens, {spacy_ms:5.1f}ms): {s_toks[:8]}')

# ── สรุปสถิติ ───────────────────────────────────────────
df_cmp = pd.DataFrame(comparison_rows)
print('\n' + '=' * 65)
print('📊 สรุป Tokenization Comparison (เฉลี่ย 5 ข้อความ)')
print('=' * 65)
print(f'  NLTK  — เฉลี่ย {df_cmp["nltk_count"].mean():>6.1f} tokens  | {df_cmp["nltk_time_ms"].mean():>6.1f} ms')
print(f'  spaCy — เฉลี่ย {df_cmp["spacy_count"].mean():>6.1f} tokens  | {df_cmp["spacy_time_ms"].mean():>6.1f} ms')
print()
print('📌 ข้อสังเกต:')
print('  • NLTK  → regex-based tokenizer, เร็วกว่า, เหมาะข้อมูลขนาดใหญ่')
print('  • spaCy → statistical model, ใช้ lemmatization (running→run)')
print('           ทำให้ vocab เล็กกว่า และรู้จัก POS/NER ในตัว')
print('  • ความแตกต่างหลัก: spaCy แยก punctuation ชัดกว่า และ')
print('    lemmatize ทำให้ token count น้อยกว่า NLTK โดยทั่วไป')

🔤 Tokenization Comparison: NLTK vs spaCy

[1] Three Top Dividend Stocks To Consider. As February begins, U.S. stock 
    NLTK  ( 39 tokens,   0.6ms): ['three', 'top', 'dividend', 'stocks', 'consider', 'february', 'begins', 'stock']
    spaCy ( 36 tokens,  17.1ms): ['dividend', 'stock', 'consider', 'february', 'begin', 'stock', 'index', 'show']

[2] Dow Jones Futures Fall After S&P 500 Hits Resistance; Carvana Dives, W
    NLTK  ( 27 tokens,   0.4ms): ['dow', 'jones', 'futures', 'fall', 'p', 'hits', 'resistance', 'carvana']
    spaCy ( 24 tokens,  11.8ms): ['dow', 'jones', 'futures', 'fall', 'hits', 'resistance', 'carvana', 'dives']

[3] Here's Why I Wouldn't Touch Medical Properties Trust With a 10 Foot Po
    NLTK  ( 16 tokens,   0.3ms): ['would', 'touch', 'medical', 'properties', 'trust', 'foot', 'pole', 'medical']
    spaCy ( 15 tokens,  10.8ms): ['touch', 'medical', 'properties', 'trust', 'foot', 'pole', 'medical', 'properties']

[4] Tech Qualms, Walmart Outlook Damp Wall Street Pr

In [18]:
# ══════════════════════════════════════════════════════════
#  Apply Tokenization ทั้ง complete_dataset
# ══════════════════════════════════════════════════════════

print('⏳ Apply tokenization บน complete_dataset ทั้งหมด...')
print(f'   จำนวน: {len(df_complete):,} รายการ')

df_complete['tokens_nltk']  = df_complete['Sentence'].apply(lambda x: tok_nltk(x[:3000]))
df_complete['tokens_spacy'] = df_complete['Sentence'].apply(lambda x: tok_spacy(x[:3000]))
df_complete['count_nltk']   = df_complete['tokens_nltk'].apply(len)
df_complete['count_spacy']  = df_complete['tokens_spacy'].apply(len)

print()
print('📊 สถิติ Token Count (ทั้ง Dataset):')
print(f'   NLTK  — เฉลี่ย {df_complete["count_nltk"].mean():.1f}  | min {df_complete["count_nltk"].min()}  | max {df_complete["count_nltk"].max()}')
print(f'   spaCy — เฉลี่ย {df_complete["count_spacy"].mean():.1f} | min {df_complete["count_spacy"].min()} | max {df_complete["count_spacy"].max()}')

print()
print('--- ตัวอย่าง 3 แถว ---')
df_complete[['Sentence', 'count_nltk', 'count_spacy']].head(3)

⏳ Apply tokenization บน complete_dataset ทั้งหมด...
   จำนวน: 6,095 รายการ

📊 สถิติ Token Count (ทั้ง Dataset):
   NLTK  — เฉลี่ย 14.5  | min 1  | max 119
   spaCy — เฉลี่ย 14.2 | min 0 | max 115

--- ตัวอย่าง 3 แถว ---


,Sentence,count_nltk,count_spacy
0,Three Top Dividend Stocks To Consider. As Febr...,39,36
1,Dow Jones Futures Fall After S&P 500 Hits Resi...,27,24
2,Here's Why I Wouldn't Touch Medical Properties...,16,15


In [19]:
# ══════════════════════════════════════════════════════════
#  CHECKLIST — พร้อมส่งต่อ Phase 2 หรือยัง?
# ══════════════════════════════════════════════════════════

print('🔍 Checklist Phase 1 — พร้อมส่งต่อ Phase 2:')
print('=' * 65)

web_ok      = os.path.exists(WEB_CSV)
complete_ok = os.path.exists(COMPLETE_CSV)

checks = {
    f'clean_text_(Web_Scraping).csv บันทึกแล้ว ({len(df_web):,} rows)':
        web_ok,
    f'complete_dataset.csv บันทึกแล้ว ({len(df_complete):,} rows)':
        complete_ok,
    'complete_dataset มี column "Sentence" เท่านั้น (ก่อน tokenize)':
        'Sentence' in df_complete.columns,
    'ไม่มีแถวที่ Sentence ว่าง':
        (df_complete['Sentence'].str.strip() == '').sum() == 0,
    'ไม่มี whitespace ซ้ำ':
        df_complete['Sentence'].str.contains(r'  ').sum() == 0,
    'ไม่มีแถวซ้ำ':
        df_complete[['Sentence']].duplicated().sum() == 0,
    'มีคอลัมน์ tokens_nltk':
        'tokens_nltk'  in df_complete.columns,
    'มีคอลัมน์ tokens_spacy':
        'tokens_spacy' in df_complete.columns,
    'ไม่มีไฟล์ raw_news.csv / processed_news.csv ใน output_data':
        not any(os.path.exists(f'{OUT_DIR}/{f}')
                for f in ['raw_news.csv', 'processed_news.csv']),
}

all_pass = True
for check, passed in checks.items():
    icon = '✅' if passed else '❌'
    print(f'  {icon} {check}')
    if not passed:
        all_pass = False

print('=' * 65)
if all_pass:
    print('\n✅ ผ่านทุกเงื่อนไข!')
    print(f'   โหลดใน Phase 2 ด้วย: pd.read_csv("{COMPLETE_CSV}")')
else:
    print('\n⚠️  มีบางเงื่อนไขที่ยังไม่ผ่าน ตรวจสอบด้านบนก่อน')

print()
print(f'📁 ไฟล์ใน {OUT_DIR}/:')
for f in sorted(os.listdir(OUT_DIR)):
    if f.endswith('.csv') and not f.startswith('_'):
        size = os.path.getsize(f'{OUT_DIR}/{f}') / 1024
        print(f'   ✅ {f}  ({size:.1f} KB)')

🔍 Checklist Phase 1 — พร้อมส่งต่อ Phase 2:
  ✅ clean_text_(Web_Scraping).csv บันทึกแล้ว (839 rows)
  ✅ complete_dataset.csv บันทึกแล้ว (6,095 rows)
  ✅ complete_dataset มี column "Sentence" เท่านั้น (ก่อน tokenize)
  ✅ ไม่มีแถวที่ Sentence ว่าง
  ✅ ไม่มี whitespace ซ้ำ
  ✅ ไม่มีแถวซ้ำ
  ✅ มีคอลัมน์ tokens_nltk
  ✅ มีคอลัมน์ tokens_spacy
  ✅ ไม่มีไฟล์ raw_news.csv / processed_news.csv ใน output_data

✅ ผ่านทุกเงื่อนไข!
   โหลดใน Phase 2 ด้วย: pd.read_csv("output_data/complete_dataset.csv")

📁 ไฟล์ใน output_data/:
   ✅ clean_text_(Web_Scraping).csv  (322.1 KB)
   ✅ complete_dataset.csv  (923.7 KB)
